In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [49]:
import numpy as np
import inspect
import types
import pickle

from typing import Generator
import itertools

from test import\
    test_eq,\
    test_is,\
    test_fail,\
    is_iter,\
    test_ne

from core import\
    FixSigMeta,\
    PrePostInitMeta,\
    BaseObj,\
    NewChkMeta,\
    BypassNewMeta,\
    patch_to,\
    patch,\
    patch_property,\
    use_kwargs,\
    delegates,\
    funcs_kwargs,\
    method,\
    runtime_check,\
    add_docs,\
    GetAttr,\
    delegate_attr,\
    coll_repr,\
    mask2idxs,\
    cycle,\
    zip_cycle,\
    L

## Metaclasses 

In [3]:
class _T(metaclass=PrePostInitMeta):
    def __pre_init__(self):
        self.a = 0
        assert self.a == 0
    def __init__(self):
        self.a += 1
        assert self.a == 1
    def __post_init__(self):
        self.a += 1
        assert self.a == 2
        
t = _T()
test_eq(t.a, 2)

In [4]:
t = _T()
isinstance(t, _T)

True

In [5]:
class _T(metaclass=NewChkMeta):
    "Testing"
    def __init__(self, o=None, b=1):
        self.foo = getattr(o, "foo", 0) + 1
        self.b = b
        
class _T2:
    "Testing 2"
    def __init__(self, o):
        self.foo = getattr(o, "foo", 0) + 1
        
t = _T(1)
test_eq(t.foo, 1)
t2 = _T(t)
test_eq(t2.foo, 1)
test_is(t, t2)
t3 = _T(t, b=2)
test_eq(t3.b, 2)
assert not t3 is t

t = _T2(1)
test_eq(t.foo, 1)
t2 = _T2(t)
test_eq(t2.foo,2)
assert not t2 is t

test_eq(_T.__doc__, "Testing")
test_eq(str(inspect.signature(_T)), "(self, o=None, b=1)")

In [6]:
class T0: pass
class _T(T0, metaclass=BypassNewMeta):
    _bypass_type=T0
    def __init__(self, x): self.x=x
        
t = T0()
t.a = 1
t2 = _T(t)
test_eq(type(t2), _T)
test_eq(t2.a, 1)
test_is(t2, t)
t = _T(2)
t.x = 2

## Foundational functions

In [7]:
class _T3(int): pass

@patch_to(_T3)
def func1(x, a): return x+a

t = _T3(1)
test_eq(t.func1(2), 3)

In [8]:
class _T4(int): pass
@patch_to((_T3, _T4))
def func2(x, a): return x+2*a

t = _T3(1)
test_eq(t.func2(1), 3)
t = _T4(1)
test_eq(t.func2(1), 3)

In [9]:
def foo(val:str)->None:
    return "hi {val}"

foo.__annotations__.values()

dict_values([<class 'str'>, None])

In [10]:
@patch
def func(x:_T3, a):
    "test"
    return x+2

t = _T3(1)
test_eq(t.func(2), 3)
test_eq(t.func.__qualname__, "_T3.func")

In [11]:
@patch
def func3(x:(_T3, _T4), a):
    "test"
    return x+2*a

t = _T3(1)
test_eq(t.func3(2), 5)
test_eq(t.func3.__qualname__, "_T3.func3")
t = _T4(1)
test_eq(t.func3(2), 5)
test_eq(t.func3.__qualname__, "_T4.func3")

In [12]:
inspect.signature(t.func3)

<Signature (a)>

In [13]:
class foo:
    def bar(self, x):
        return x+1
inspect.signature(foo().bar)

<Signature (x)>

In [14]:
@patch_property
def prop(x:_T3): return x+1

t = _T3(1)
test_eq(t.prop, 2)

In [15]:
inspect.Parameter("car", inspect.Parameter.KEYWORD_ONLY)

<Parameter "car">

In [16]:
def test_sig(func, b): test_eq(str(inspect.signature(func)), b)

In [17]:
@use_kwargs(["y", "z"])
def foo(a, b=1, **kwargs): pass
test_sig(foo, '(a, b=1, *, y=None, z=None)')

@use_kwargs(["y", "z"], keep=True)
def foo(a, *args, b=1, **kwargs): pass
test_sig(foo, '(a, *args, b=1, y=None, z=None, **kwargs)')

In [18]:
def basefoo(e, c=2): pass

@delegates(basefoo)
def foo(a, b=1, **kwargs): pass
test_sig(foo, '(a, b=1, c=2)')

@delegates(basefoo, keep=True)
def foo(a, b=1, **kwargs): pass
test_sig(foo, '(a, b=1, c=2, **kwargs)')

In [19]:
@funcs_kwargs
class T:
    _methods=["b"]
    def __init__(self, f=1, **kwargs): assert not kwargs
    def a(self): return 1
    def b(self): return 2
    
t = T()
test_eq(t.a(), 1)
test_eq(t.b(), 2)
t = T(b=lambda:3)
test_eq(t.b(), 3)
test_sig(T, '(f=1, *, b=None)')
test_fail(lambda: T(a = lambda:3))

@method
def _f(self, a=1): return a+1
t = T(b = _f)
test_eq(t.b(2), 3)

class T2(T):
    def __init__(self, a):
        super().__init__(b = lambda: 3)
        self.a = a
t = T2(a=1)
test_eq(t.b(), 3)
test_sig(T2, '(a)')

def _g(a=1): return a+1
class T3(T): b = staticmethod(_g)
t = T3()
test_eq(t.b(2), 3)

In [20]:
@runtime_check
def test_check(a:int=1): return a

test_eq(test_check(2), 2)
test_eq(test_check(), 1)
test_fail(lambda: test_check("a"), contains='"a" must be int')

In [21]:
t = pickle.loads(pickle.dumps(test_check))
test_eq(t(2),2)
test_eq(t(),1)

In [22]:
def is_listy(x): return isinstance(x, (list, tuple, Generator))

In [23]:
class _T:
    def f(self): pass
    @classmethod
    def g(cls): pass
add_docs(_T, "a", f="f", g="g")

test_eq(_T.__doc__, "a")
test_eq(_T.f.__doc__, "f")
test_eq(_T.g.__doc__, "g")

In [24]:
assert is_iter([1])
assert not is_iter(np.array(1))
assert is_iter(np.array([1,2]))
assert (o for o in range(3))

## GetAttr 

In [32]:
class _C(GetAttr):
    _xtra = ['lower']
    def __init__(self, a): self.default = a
    def foo(self): noop
    def test(self): return "low"
        
t = _C("Hi")
test_eq(t.lower(), "hi")
test_fail(lambda: t.upper())
assert "lower" in dir(t)

In [35]:
t._xtra()

TypeError: 'list' object is not callable

In [26]:
class _C:
    f = "Hi"
    def __getattr__(self, k): return delegate_attr(self, k, "f")
    
t = _C()
test_eq(t.lower(), "hi")

## L- 

In [27]:
coll_repr(range(1000), 10)

'(#1000) [0,1,2,3,4,5,6,7,8,9...]'

In [28]:
coll_repr(range(5), 10)

'(#5) [0,1,2,3,4]'

In [29]:
test_eq(mask2idxs([False, True, False, True]), [1,3])
test_eq(mask2idxs(np.array([1, 2, 3])), [1, 2, 3])

In [30]:
test_eq(itertools.islice(cycle([1,2,3]), 5), [1,2,3,1,2])
test_eq(itertools.islice(cycle([]),3), [None]*3)
test_eq(itertools.islice(cycle(None), 3), [None]*3)
test_eq(itertools.islice(cycle(1), 3), [1,1,1])

In [31]:
test_eq(zip_cycle([1,2,3,4], list("abc")), [(1, "a"), (2, "b"), (3, "c"), (4, "a")])

In [36]:
type(range(12))

range

In [145]:
t = L(range(12))
test_eq(t, list(range(12)))
test_ne(t, list(range(11)))
t.reverse() # uses list methods (clever!)
test_eq(t[0], 11)
t[3] = "h"
t[3,5] = ("j", "k") # __setitem__ modified
test_eq(t[3,5], ["j", "k"])
test_eq(t, L(t))
test_eq(L(L(1,2), [3,4]), ([1,2], [3,4])) # arg is tuple ([3,4],)